In [ ]:
%pip install opencv-python mediapipe numpy pyttsx3

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
import winsound   


def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    return (A + B) / (2.0 * C)


mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)

LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]


EAR_THRESHOLD = 0.25
MIN_CLOSE_TIME = 2.0       
BEEP_INTERVAL = 1.0       

eye_closed_since = None
last_beep_time = 0

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    ear = None
    left_eye_pts, right_eye_pts = [], []

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            h, w, _ = frame.shape

            for idx in LEFT_EYE:
                left_eye_pts.append([
                    int(face_landmarks.landmark[idx].x * w),
                    int(face_landmarks.landmark[idx].y * h)
                ])
            for idx in RIGHT_EYE:
                right_eye_pts.append([
                    int(face_landmarks.landmark[idx].x * w),
                    int(face_landmarks.landmark[idx].y * h)
                ])

            left_eye = np.array(left_eye_pts)
            right_eye = np.array(right_eye_pts)

            ear = (eye_aspect_ratio(left_eye) +
                   eye_aspect_ratio(right_eye)) / 2

          
            cv2.polylines(frame, [left_eye], True, (0, 0, 255), 2)
            cv2.polylines(frame, [right_eye], True, (0, 0, 255), 2)

    now = time.time()

    if ear is not None and ear < EAR_THRESHOLD:
        if eye_closed_since is None:
            eye_closed_since = now

        closed_time = now - eye_closed_since

        if closed_time >= MIN_CLOSE_TIME:
            
            cv2.rectangle(frame, (0, 0),
                          (frame.shape[1], 90),
                          (0, 0, 255), -1)
            cv2.putText(frame, "DROWSY! WAKE UP!",
                        (40, 60),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1.8, (255, 255, 255), 4)

           
            if now - last_beep_time >= BEEP_INTERVAL:
                winsound.Beep(1200, 300)  
                last_beep_time = now
    else:
        eye_closed_since = None
        last_beep_time = 0

    cv2.imshow("Driver Drowsiness Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
